In [1]:
import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import glob
import sys
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.manifold import TSNE
from sklearn import preprocessing
import pandas as pd
from multiprocessing import Process# this is used for multithreading
import multiprocessing
import codecs# this is used for file operations 
import random as r
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
train_benign_df = pd.read_csv("./csv_files/train_benign_asmfile.txt")
train_benign_df.drop(train_benign_df.columns[-1], axis=1, inplace=True)
train_benign_df.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
train_benign_df['label'] = 0

train_malware_df = pd.read_csv("./csv_files/train_malware_asmfile.txt")
train_malware_df.drop(train_malware_df.columns[-1], axis=1, inplace=True)
train_malware_df.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
train_malware_df['label'] = 1

test_benign_df = pd.read_csv("./csv_files/test_benign_asmfile.txt")
test_benign_df.drop(test_benign_df.columns[-1], axis=1, inplace=True)
test_benign_df.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
test_benign_df['label'] = 0

test_malware_df = pd.read_csv("./csv_files/test_malware_asmfile.txt")
test_malware_df.drop(test_malware_df.columns[-1], axis=1, inplace=True)
test_malware_df.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
test_malware_df['label'] = 1

valid_benign = pd.read_csv("./csv_files/valid_benign_asmfile.txt")
valid_benign.drop(valid_benign.columns[-1], axis=1, inplace=True)
valid_benign.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
valid_benign['label'] = 0

valid_malware = pd.read_csv("./csv_files/valid_malware_asmfile.txt")
valid_malware.drop(valid_malware.columns[-1], axis=1, inplace=True)
valid_malware.set_axis(['ID', 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword'], axis='columns', inplace=True)
valid_malware['label'] = 1


In [3]:
import sklearn
df_list = [train_benign_df, train_malware_df, test_benign_df, test_malware_df, valid_benign, valid_malware]
full_df = pd.concat(df_list, ignore_index=True, axis=0)
full_shuffled_df = sklearn.utils.shuffle(full_df)
full_shuffled_df

,ID,jmp,mov,retf,push,pop,xor,retn,nop,sub,...,ebx,ecx,edi,ebp,esp,eip,.dll,std::,:dword,label
1680,3f653f661b8a07279cd5aeb40d7d18b1,571,7030,0,5140,1176,588,401,0,468,...,0,0,0,0,0,0,57,18,45,1
626,2bbd5b91fe31b0fe983ef181d13bf46c,327,3341,0,2143,593,155,411,0,158,...,0,0,0,0,0,0,2,332,0,0
2651,4b10563cf25f64ad083b494b1b42e203,259,2412,0,1583,583,353,134,0,176,...,0,0,0,0,0,0,3,0,75,1
187,1ae1b3d140c29e514ca09facf0e3439f,84,1335,0,799,422,75,221,2,66,...,0,0,0,0,0,0,0,13,0,0
1716,1bd94962bd242d4f3bcc08544c0a5cba,174,1284,0,1183,221,93,79,2,80,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,62d2bc7109f57da7c722ca136ba7273b,40,165,0,207,53,43,40,0,4,...,0,0,0,0,0,0,0,0,0,0
756,3ca853cdea4d75335b4f1d5bb68b4677,2273,12188,0,10726,1859,989,945,0,331,...,0,0,0,14,0,0,7,256,2,0
1250,2e4e4a4df737e891fe729abff820bc73,33,381,0,224,68,47,31,2,38,...,0,0,0,0,0,0,0,0,0,1
1637,3f9e0f93c6ba69d50b25a08598dc2b95,82,335,2,689,12,7,9,0,27,...,0,0,0,0,0,0,0,0,0,1


In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
NORMALIZE_VALID_COLUMNS = [ 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword']

full_shuffled_df[NORMALIZE_VALID_COLUMNS] = scaler.fit_transform(full_shuffled_df[NORMALIZE_VALID_COLUMNS])
full_shuffled_df[NORMALIZE_VALID_COLUMNS].astype('float')
full_shuffled_df

,ID,jmp,mov,retf,push,pop,xor,retn,nop,sub,...,ebx,ecx,edi,ebp,esp,eip,.dll,std::,:dword,label
1680,3f653f661b8a07279cd5aeb40d7d18b1,0.005476,0.004353,0.000000,0.008495,0.005130,0.003766,0.004681,0.000000,0.004206,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.168639,0.000867,0.001525,1
626,2bbd5b91fe31b0fe983ef181d13bf46c,0.003136,0.002064,0.000000,0.003542,0.002587,0.000993,0.004797,0.000000,0.001420,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.005917,0.015995,0.000000,0
2651,4b10563cf25f64ad083b494b1b42e203,0.002484,0.001487,0.000000,0.002616,0.002543,0.002261,0.001564,0.000000,0.001582,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.008876,0.000000,0.002541,1
187,1ae1b3d140c29e514ca09facf0e3439f,0.000806,0.000819,0.000000,0.001321,0.001841,0.000480,0.002580,0.000086,0.000593,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000626,0.000000,0
1716,1bd94962bd242d4f3bcc08544c0a5cba,0.001669,0.000787,0.000000,0.001955,0.000964,0.000596,0.000922,0.000086,0.000719,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,62d2bc7109f57da7c722ca136ba7273b,0.000384,0.000093,0.000000,0.000342,0.000231,0.000275,0.000467,0.000000,0.000036,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0
756,3ca853cdea4d75335b4f1d5bb68b4677,0.021800,0.007553,0.000000,0.017728,0.008110,0.006334,0.011030,0.000000,0.002974,...,0.0,0.0,0.0,0.097222,0.0,0.0,0.020710,0.012333,0.000068,0
1250,2e4e4a4df737e891fe729abff820bc73,0.000317,0.000227,0.000000,0.000370,0.000297,0.000301,0.000362,0.000086,0.000341,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1
1637,3f9e0f93c6ba69d50b25a08598dc2b95,0.000786,0.000199,0.001432,0.001139,0.000052,0.000045,0.000105,0.000000,0.000243,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(full_shuffled_df[NORMALIZE_VALID_COLUMNS],full_shuffled_df['label'], stratify=full_shuffled_df['label'], test_size=0.2, random_state=42)

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [13]:
def ACCURACY(true, pred):   
    score = np.mean(true==pred)
    return score

In [14]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [23]:
data = full_shuffled_df.drop('ID', axis=1)
data.head()

,jmp,mov,retf,push,pop,xor,retn,nop,sub,inc,...,ebx,ecx,edi,ebp,esp,eip,.dll,std::,:dword,label
1680,0.005476,0.004353,0.0,0.008495,0.005130,0.003766,0.004681,0.000000,0.004206,0.004917,...,0.0,0.0,0.0,0.0,0.0,0.0,0.168639,0.000867,0.001525,1
626,0.003136,0.002064,0.0,0.003542,0.002587,0.000993,0.004797,0.000000,0.001420,0.000632,...,0.0,0.0,0.0,0.0,0.0,0.0,0.005917,0.015995,0.000000,0
2651,0.002484,0.001487,0.0,0.002616,0.002543,0.002261,0.001564,0.000000,0.001582,0.002613,...,0.0,0.0,0.0,0.0,0.0,0.0,0.008876,0.000000,0.002541,1
187,0.000806,0.000819,0.0,0.001321,0.001841,0.000480,0.002580,0.000086,0.000593,0.000084,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000626,0.000000,0
1716,0.001669,0.000787,0.0,0.001955,0.000964,0.000596,0.000922,0.000086,0.000719,0.002107,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1


In [15]:
def kfold(model, train, scale = False):
    cv_accuracy = []
    cv = StratifiedKFold(n_splits=5)
    
    n_iter = 0
    
    for t, v in cv.split(train, train['label']):
        
        train_cv = train.iloc[t] # 훈련용
        val_cv = train.iloc[v] # 검증용 분리

        train_X = train_cv.drop('label', axis=1)
        train_y = train_cv['label']

        val_X = val_cv.drop('label', axis=1)
        val_y = val_cv['label']
            
        model.fit(train_X, train_y)
        score = ACCURACY(val_y, model.predict(val_X))
        
        cv_accuracy.append(score)
        n_iter += 1
    return np.mean(cv_accuracy)

In [25]:
# for LGBM error
import re
data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [26]:
models = [
    KNeighborsClassifier(),
    LogisticRegression(),
    DecisionTreeClassifier(),
    RandomForestClassifier(max_depth=18, min_samples_leaf=2, min_samples_split=2, n_estimators=40, random_state=0, n_jobs=-1),
    GradientBoostingClassifier(),
    XGBClassifier(eval_metric = 'logloss', 
                              max_depth = 5, 
                              min_child_weight = 3, 
                               gamma = 3, 
                               colsample_bytree = 0.5, 
                               n_estimators=700),
    LGBMClassifier(n_estimators=600, max_bin=400, num_leaves=24),
    CatBoostClassifier(silent=True, depth=6, l2_leaf_reg=7, learning_rate=0.1, n_estimators=500),
    ExtraTreesClassifier(max_depth=25, n_estimators=320)
]

print('각 모델마다의 Kfold')
for model in models:
  print(f'{type(model).__name__} score: {kfold(model, data)}')

각 모델마다의 Kfold
KNeighborsClassifier score: 0.8747347093183553
LogisticRegression score: 0.6027128577016049
DecisionTreeClassifier score: 0.8833227537653643
RandomForestClassifier score: 0.9241188502253769
GradientBoostingClassifier score: 0.8983515109546618
XGBClassifier score: 0.9216207898128378
LGBMClassifier score: 0.923050634453485
CatBoostClassifier score: 0.9219747244503432
ExtraTreesClassifier score: 0.9291354890004552


### Model Tuning

성능이 가장 잘 나온 상위 4개만 사용, 0.92로 많이 차이 나지 않는 경우 많이 쓰이는 모델로 사용  
Randomforest, LGBMClassifier, XGBClassifier, CatBoost

#### RandomForest

In [27]:
params = { 'n_estimators' : [10, 20, 30, 40, 50, 60, 70, 80],
           'max_depth' : [14, 16, 18, 20, 22],
           'min_samples_leaf' : [2, 3, 4],
           'min_samples_split' : [1, 2, 3, 4]
            }

In [29]:
train = full_shuffled_df.drop(['ID','label'], axis=1)
target = full_shuffled_df['label']

model_RFC = RandomForestClassifier()
grid_cv_RFC = GridSearchCV(model_RFC, param_grid = params, cv=5, n_jobs = -1)
grid_cv_RFC.fit(train, target)
print('최적 하이퍼 파라미터: ', grid_cv_RFC.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_RFC.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 14, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 40}
최고 예측 정확도: 0.9288


#### XGBClassifier

In [36]:
import warnings
warnings.filterwarnings(action="ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
model_XGB = XGBClassifier(eval_metric='logloss', silent = True)

param_grid={'booster' :['gbtree'],
                 'silent':[True],
                 'max_depth':[5,6,8],
                 'min_child_weight':[1,3,5],
                 'gamma':[0,1,2,3],
                 'nthread':[4],
                 'colsample_bytree':[0.5,0.8],
                 'colsample_bylevel':[0.9],
                 'n_estimators':[50],
                 'objective':['binary:logistic'],
                 'random_state':[2]}

grid_cv_XGB=GridSearchCV(model_XGB, param_grid=param_grid, cv=5 , n_jobs=-1)
grid_cv_XGB.fit(train, target)
print('최적 하이퍼 파라미터: ', grid_cv_XGB.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_XGB.best_score_))

최적 하이퍼 파라미터:  {'booster': 'gbtree', 'colsample_bylevel': 0.9, 'colsample_bytree': 0.8, 'gamma': 0, 'max_depth': 8, 'min_child_weight': 3, 'n_estimators': 50, 'nthread': 4, 'objective': 'binary:logistic', 'random_state': 2, 'silent': True}
최고 예측 정확도: 0.9266

#### LGBM

In [39]:
model_LGBM = LGBMClassifier()

gridParams = {
    'learning_rate': [0.005, 0.01, 0.1],
    'n_estimators': [100, 500, 1000],
    'num_leaves': [12, 16, 20], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['dart'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'max_bin':[300, 600], # large max_bin helps improve accuracy but might slow down training progress
    }

grid_cv_LGBM = GridSearchCV(model_LGBM, param_grid=gridParams, cv=3 , n_jobs=-1)
grid_cv_LGBM.fit(data, target)
print('최적 하이퍼 파라미터: ', grid_cv_LGBM.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_LGBM.best_score_))

최적 하이퍼 파라미터:  {'boosting_type': 'dart', 'learning_rate': 0.005, 'max_bin': 300, 'n_estimators': 100, 'num_leaves': 12, 'objective': 'binary'}
최고 예측 정확도: 1.0000


#### CatBoostClassifier

In [40]:
model_CAT = CatBoostClassifier(silent = True)

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_cv_CAT = GridSearchCV(model_CAT, param_grid=grid, cv=3 , n_jobs=-1)
grid_cv_CAT.fit(data, target)
print('최적 하이퍼 파라미터: ', grid_cv_CAT.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_CAT.best_score_))

최적 하이퍼 파라미터:  {'depth': 4, 'l2_leaf_reg': 1, 'learning_rate': 0.03}
최고 예측 정확도: 1.0000


### Soft Voting

In [45]:
INCLUDE_LABEL = [ 'jmp', 'mov', 'retf', 'push', 'pop', 'xor', 'retn', 'nop', 'sub', 'inc', 'dec', 'add','imul', 'xchg', 'or', 'shr', 'cmp', 'call', 'shl', 'ror', 'rol', 'jnb','jz','rtn','lea','movzx', 'edx','esi','eax','ebx','ecx','edi','ebp','esp','eip', '.dll', 'std::', ':dword', 'label']

X_train, X_test, y_train, y_test = train_test_split(full_shuffled_df[INCLUDE_LABEL],full_shuffled_df['label'], stratify=full_shuffled_df['label'], test_size=0.2, random_state=42)

In [48]:
train = X_train.drop('label', axis=1)
target = X_train['label']
# for LGBM error
data = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
best_model_XGB = XGBClassifier(eval_metric = 'logloss', 
                              booster = 'gbtree',
                              colsample_bylevel=0.9,
                              colsample_bytree=0.8,
                              silent = True, 
                              max_depth = 8, 
                              min_child_weight = 3, 
                               gamma = 0, 
                               n_estimators=50,
                               nthread=4,
                               objective='binary:logistic',
                               random_state=2,
                               )
best_model_LGBM = LGBMClassifier(boosting_type='dart',learning_rate=0.005 ,n_estimators=100, max_bin=300, num_leaves=12, objective='binary')
best_model_CAT = CatBoostClassifier(silent=True, depth=4, l2_leaf_reg=1, learning_rate=0.03)

In [49]:
from sklearn.ensemble import VotingClassifier
softVoting_model = VotingClassifier(estimators=[('XGB', best_model_XGB), ('LGBM', best_model_LGBM), ('CAT', best_model_CAT)], voting='soft')
softVoting_model.fit(data, target)

[16:52:18] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1644955194972/work/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




VotingClassifier(estimators=[('XGB',
                              XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=0.9,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8,
                                            enable_categorical=False,
                                            eval_metric='logloss', gamma=0,
                                            gpu_id=None, importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=8,
                                            min_child_weight=3, missing=nan,
                                            mon...
                                            nthread=4, num_parallel_tree=None,
                                    

In [52]:
X_test = X_test.drop('label', axis=1)

In [54]:
from sklearn.metrics import classification_report, confusion_matrix
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

soft_pred = softVoting_model.predict(X_test)
print(classification_report(y_test, soft_pred))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93       280
           1       0.94      0.91      0.93       279

    accuracy                           0.93       559
   macro avg       0.93      0.93      0.93       559
weighted avg       0.93      0.93      0.93       559



### PCA 진행 후 비교

In [97]:
from sklearn.decomposition import PCA

skpca = PCA(n_components=30)
X_pca = skpca.fit_transform(data)
print('Variance sum : ', skpca.explained_variance_ratio_.cumsum()[-1]) # 주어진 개수가 전체 데이터의 분산을 얼마만큼 설명 가능한지에 대한 수치

Variance sum :  0.9979631720960535


In [99]:
params = { 'n_estimators' : [10, 20, 30, 40, 50, 60, 70, 80],
           'max_depth' : [14, 16, 18, 20, 22],
           'min_samples_leaf' : [2, 3, 4],
           'min_samples_split' : [1, 2, 3, 4]
            }

model_RFC = RandomForestClassifier()
grid_cv_RFC = GridSearchCV(model_RFC, param_grid = params, cv=5, n_jobs = -1)
grid_cv_RFC.fit(X_pca, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_RFC.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_RFC.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 18, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 50}
최고 예측 정확도: 0.9244


In [ ]:
model_XGB = XGBClassifier(eval_metric='logloss', silent = True)

param_grid={'booster' :['gbtree'],
                 'silent':[True],
                 'max_depth':[5,6,8],
                 'min_child_weight':[1,3,5],
                 'gamma':[0,1,2,3],
                 'nthread':[4],
                 'colsample_bytree':[0.5,0.8],
                 'colsample_bylevel':[0.9],
                 'n_estimators':[50],
                 'objective':['binary:logistic'],
                 'random_state':[2]}

grid_cv_XGB=GridSearchCV(model_XGB, param_grid=param_grid, cv=5 , n_jobs=-1)
grid_cv_XGB.fit(X_pca, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_XGB.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_XGB.best_score_))

최적 하이퍼 파라미터:  {'booster': 'gbtree', 'colsample_bylevel': 0.9, 'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 50, 'nthread': 4, 'objective': 'binary:logistic', 'random_state': 2, 'silent': True}
최고 예측 정확도: 0.9199

In [101]:
model_LGBM = LGBMClassifier()

gridParams = {
    'learning_rate': [0.005, 0.01, 0.1],
    'n_estimators': [100, 500, 1000],
    'num_leaves': [12, 16, 20], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['dart'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'max_bin':[300, 600], # large max_bin helps improve accuracy but might slow down training progress
    }

grid_cv_LGBM = GridSearchCV(model_LGBM, param_grid=gridParams, cv=3 , n_jobs=-1)
grid_cv_LGBM.fit(X_pca, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_LGBM.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_LGBM.best_score_))

KeyboardInterrupt: 

In [ ]:
model_CAT = CatBoostClassifier(silent = True)

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_cv_CAT = GridSearchCV(model_CAT, param_grid=grid, cv=3 , n_jobs=-1)
grid_cv_CAT.fit(X_pca, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_CAT.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_CAT.best_score_))

최적 하이퍼 파라미터:  {'depth': 10, 'l2_leaf_reg': 3, 'learning_rate': 0.1}
최고 예측 정확도: 0.9087


In [ ]:
best_model_RF = RandomForestClassifier(max_depth=16, min_samples_leaf=2, min_samples_split=2, n_estimators=50)
best_model_XGB = XGBClassifier(eval_metric = 'logloss', 
                              booster = 'gbtree',
                              colsample_bylevel=0.9,
                              colsample_bytree=0.5,
                              silent = True, 
                              max_depth = 8, 
                              min_child_weight = 1, 
                               gamma = 0, 
                               n_estimators=50,
                               nthread=4,
                               objective='binary:logistic',
                               random_state=2,
                               )
best_model_LGBM = LGBMClassifier(boosting_type='dart',learning_rate=0.1 ,n_estimators=1000, max_bin=300, num_leaves=16, objective='binary')
best_model_CAT = CatBoostClassifier(silent=True, depth=10, l2_leaf_reg=3, learning_rate=0.1)

In [ ]:
softVoting_model_pca = VotingClassifier(estimators=[('RF', best_model_RF),('XGB', best_model_XGB), ('LGBM', best_model_LGBM), ('CAT', best_model_CAT)], voting='soft')
softVoting_model_pca.fit(X_pca, y_train)

[18:09:45] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1644955194972/work/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




VotingClassifier(estimators=[('RF',
                              RandomForestClassifier(max_depth=16,
                                                     min_samples_leaf=2,
                                                     n_estimators=50)),
                             ('XGB',
                              XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=0.9,
                                            colsample_bynode=None,
                                            colsample_bytree=0.5,
                                            enable_categorical=False,
                                            eval_metric='logloss', gamma=0,
                                            gpu_id=None, importance_type=None,
                                            interaction_constraints=None,
                                            learni...
                                            nthread=4, num_parallel_tree=None,
    

In [ ]:
X_test_pca = skpca.transform(X_test)

In [ ]:
soft_pred_pca = softVoting_model_pca.predict(X_test_pca)
print(classification_report(y_test, soft_pred_pca))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90       280
           1       0.92      0.87      0.90       279

    accuracy                           0.90       559
   macro avg       0.90      0.90      0.90       559
weighted avg       0.90      0.90      0.90       559

